<a href="https://colab.research.google.com/github/nikxlvii/pytorch/blob/main/breast_cancer_diagnosis_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

data = pd.read_csv('idc.csv')
data.head()

,image_id,patient_id,x,y,target
0,10286_idx5_x1251_y351_class1.png,10286,1251,351,1
1,14079_idx5_x2151_y1401_class0.png,14079,2151,1401,0
2,12749_idx5_x1451_y701_class0.png,12749,1451,701,0
3,14157_idx5_x1651_y301_class0.png,14157,1651,301,0
4,10269_idx5_x1351_y651_class1.png,10269,1351,651,1


In [4]:
# we will split the data by patient id

patient_ids = data.patient_id.unique()

from sklearn.model_selection import train_test_split

train_ids, val_test_ids = train_test_split(patient_ids, test_size = 0.4, random_state = 24)
val_ids, test_ids = train_test_split(val_test_ids, test_size = 0.5, random_state = 24)

In [7]:
train = data[data['patient_id'].isin(train_ids)].reset_index(drop=True)
val = data[data['patient_id'].isin(val_ids)].reset_index(drop=True)
test = data[data['patient_id'].isin(test_ids)].reset_index(drop=True)

print(train.head())
print(val.head())
print(test.head())

                            image_id  patient_id     x     y  target
0  14079_idx5_x2151_y1401_class0.png       14079  2151  1401       0
1   12749_idx5_x1451_y701_class0.png       12749  1451   701       0
2   14157_idx5_x1651_y301_class0.png       14157  1651   301       0
3   10269_idx5_x1351_y651_class1.png       10269  1351   651       1
4  13693_idx5_x1901_y2851_class0.png       13693  1901  2851       0
                            image_id  patient_id     x     y  target
0   10286_idx5_x1251_y351_class1.png       10286  1251   351       1
1    9266_idx5_x1901_y701_class0.png        9266  1901   701       0
2   12910_idx5_x1001_y851_class1.png       12910  1001   851       1
3  12890_idx5_x2001_y1251_class0.png       12890  2001  1251       0
4   10293_idx5_x551_y1701_class0.png       10293   551  1701       0
                            image_id  patient_id     x     y  target
0    10295_idx5_x801_y951_class0.png       10295   801   951       0
1  10301_idx5_x1551_y1701_class0.p

In [8]:
import os
from PIL import Image

In [9]:
from torch.utils.data import Dataset

class PatchDataset(Dataset):
  def __init__(self,data,images_dir,transform=None):
    super().__init__()
    self.data = list(data.itertuples(name='Patch', index=False))
    self.images_dir = images_dir
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index):
    image_id, label = self.data[index].image_id, self.data[index].target
    image = Image.open(os.path.join(self.images_dir,image_id))
    image = image.convert('RGB')
    if self.transform is not None:
      image = self.transform(image)
    return image,label

In [10]:
import torchvision.transforms as transforms

Transform = transforms.Compose(
    [
        transforms.Resize((50,50)), # resizing the image to 50 x 50 pixels
        transforms.RandomHorizontalFlip(), # flips the image about the horizontal axis
        transforms.RandomVerticalFlip(), # flips the image about the vertical axis
        transforms.ToTensor(), # converts the image to a Numpy array
        transforms.Normalize(mean=[0.5,0.5,0.5],
                             std=[0.5,0.5,0.5]) # normalizes the image
    ]
)

In [ ]:
from torch.utils.data import dataloader

train_dataset = PatchDataset(df_data=train,images_dir = all_images_dir,transform = Transform)